In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import gc
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.svm import SVC

# Setup path to access modular scripts in src/
sys.path.append(str(Path.cwd().parent))
from src import config, preprocessing, evaluation, utils

# Initialize result folders for the Cross-dataset experiment
exp_paths = utils.setup_experiment_folder()
print(f"📂 Saving Cross-Dataset SVM results to: {exp_paths['root']}")


In [ ]:
print("\n--- 1. LOADING & SAMPLING TRAINING SET (CIC-IDS2017) ---")
# Load the 2017 dataset in binary mode [cite: 609, 623]
df_train, y_train_full = preprocessing.load_single_dataset_year('2017', binary_mode=True)

# Sampling to accommodate 16GB RAM 
TRAIN_SAMPLE_SIZE = 100000 
if len(df_train) > TRAIN_SAMPLE_SIZE:
    df_train = df_train.sample(n=TRAIN_SAMPLE_SIZE, random_state=config.RANDOM_STATE)
    y_train = pd.Series(y_train_full).iloc[df_train.index].values
else:
    y_train = y_train_full

print(f"Training Data Shape: {df_train.shape}")
print(f"Label Distribution:\n{pd.Series(y_train).value_counts()}")

del y_train_full
gc.collect()

In [ ]:
print("\n--- 2. PREPROCESSING & FEATURE SELECTION ---")

# Extract the Top-25 mRMR features used for the baseline [cite: 663]
all_features = config.SELECTED_FEATURES
mrmr_25_list = config.mRMR_FEATURES[:25] 
indices = [all_features.index(f) for f in mrmr_25_list]

X_train = df_train.values[:, indices]

# Fit scaler on training data only to prevent Data Leakage [cite: 515, 599]
scaler = preprocessing.get_scaler()
X_train_scaled = scaler.fit_transform(X_train)

# Save the scaler to persist parameters for the test set [cite: 599]
joblib.dump(scaler, exp_paths['models'] / "scaler_cross_svm.joblib")

del df_train
gc.collect()

In [ ]:
print("\n--- 3. TRAINING SVM ON CIC-IDS2017 ---")

# probability=True is required to generate the ROC curves for Chapter 5.7 [cite: 595, 633]
svm_model = SVC(
    kernel='rbf', 
    C=1.0, 
    probability=True, 
    random_state=config.RANDOM_STATE,
    verbose=True
)

svm_model.fit(X_train_scaled, y_train)

# Persist model for versioning [cite: 600]
joblib.dump(svm_model, exp_paths['models'] / "svm_cross_17_to_18.joblib")
print("✅ SVM Training Complete.")

In [ ]:
print("\n--- 4. LOADING & PROCESSING TEST SET (CSE-CIC-IDS2018) ---")
# Load the 2018 dataset [cite: 625, 626]
df_test, y_test_full = preprocessing.load_single_dataset_year('2018', binary_mode=True)

# Sampling test set for faster evaluation [cite: 626]
TEST_SAMPLE_SIZE = 50000 
df_test_sample = df_test.sample(n=TEST_SAMPLE_SIZE, random_state=config.RANDOM_STATE)
y_test = pd.Series(y_test_full).iloc[df_test_sample.index].values

# Apply the SAME feature indices and SCALER as the train set [cite: 514, 599]
X_test_scaled = scaler.transform(df_test_sample.values[:, indices])

print(f"Test Data Shape: {X_test_scaled.shape}")
del df_test, df_test_sample, y_test_full
gc.collect()

In [ ]:
print("\n--- 5. CROSS-DATASET EVALUATION (17 -> 18) ---")

metrics = evaluation.evaluate_model(
    svm_model, 
    X_test_scaled, 
    y_test, 
    save_dir=exp_paths['figures'],
    dataset_name="SVM Cross-Dataset (17 to 18)"
)

print("\n=== FINAL CROSS-DATASET METRICS (SVM) ===")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"MCC:      {metrics['mcc']:.4f}") # Expect a significant drop due to Distribution Shift 
print(f"Recall:   {metrics['recall_macro']:.4f}")

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import gc
import joblib
import numpy as np
import pandas as pd
from pathlib import Path

# Setup path to modular scripts
sys.path.append(str(Path.cwd().parent))
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils

# Create a unique folder for the 18-to-17 Cross-Test [cite: 594]
exp_paths = utils.setup_experiment_folder()
print(f"📂 Results will be saved to: {exp_paths['root']}")

print("\n--- 1. LOADING TRAINING SET (CSE-CIC-IDS2018) ---")
# Load 2018 data in binary mode [cite: 609]
df_train, y_train_full = preprocessing.load_single_dataset_year('2018', binary_mode=True)

# Sampling 1 million records for stable training on 16GB RAM
TRAIN_SIZE = 1000000 
if len(df_train) > TRAIN_SIZE:
    df_train = df_train.sample(n=TRAIN_SIZE, random_state=config.RANDOM_STATE)
    y_train = pd.Series(y_train_full).iloc[df_train.index].values
else:
    y_train = y_train_full

print(f"Training Data Shape: {df_train.shape}")
del y_train_full
gc.collect()

print("\n--- 2. SCALING & FEATURE SELECTION ---")
scaler = preprocessing.get_scaler()
X_train_scaled = scaler.fit_transform(df_train.values)

# Persist the scaler parameters for the 2017 test set [cite: 599]
joblib.dump(scaler, exp_paths['models'] / "scaler_2018_to_2017.joblib")

# Identify mRMR indices for Fusion [cite: 463, 545]
all_features = config.SELECTED_FEATURES
mrmr_indices = [all_features.index(f) for f in config.mRMR_FEATURES]

print("✅ Scaler fitted on 2018 and saved.")

print("\n--- 3. TRAINING AUTOENCODER ---")
L = 30 # Top-30 features for AE input [cite: 524]

# Slice input for AE
X_train_ae = X_train_scaled[:, :L]

ae_model = autoencoder.DeepAutoencoder(
    input_dim=L,
    latent_dim=config.AE_LATENT_DIM, # N=5 [cite: 536]
    hidden_layers=config.AE_HIDDEN_LAYERS
)

# Train and save AE model [cite: 600]
ae_save_path = exp_paths['models'] / "ae_model_2018.pth"
ae_model = autoencoder.train_ae(ae_model, X_train_ae, save_path=ae_save_path)

# Extract Latent Features (Z)
print("Extracting latent space features...")
X_train_latent = autoencoder.extract_features(ae_model, X_train_ae)

print("\n--- 4. FEATURE FUSION (2018) ---")
# Use the top 20 mRMR features [cite: 545]
X_train_mrmr = X_train_scaled[:, mrmr_indices]

# Combine into 25-dimensional Fusion Set [cite: 546]
X_train_fusion = np.hstack([X_train_mrmr, X_train_latent])

print(f"Final Fusion Training Shape: {X_train_fusion.shape}")

# Clear RAM for Test Set
del df_train, X_train_scaled, X_train_latent, X_train_mrmr
gc.collect()


print("\n--- 5. TRAINING CLASSIFIER (RF) ---")
rf_save_path = exp_paths['models'] / "rf_cross_18_to_17.joblib"
rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)
print("✅ Hybrid Model trained on 2018.")


print("\n--- 6. LOADING TEST SET (CIC-IDS2017) ---")
df_test, y_test = preprocessing.load_single_dataset_year('2017', binary_mode=True)
print(f"Test Set Shape: {df_test.shape}")


print("\n--- 7. PROCESSING TEST DATA ---")
# A. Apply 2018 Scaling
X_test_scaled = scaler.transform(df_test.values)

# B. Extract Latent features using 2018 AE
X_test_latent = autoencoder.extract_features(ae_model, X_test_scaled[:, :L])

# C. Fusion: 20 mRMR (from 2018 indices) + 5 Latent
X_test_fusion = np.hstack([X_test_scaled[:, mrmr_indices], X_test_latent])

print(f"Test Fusion Shape: {X_test_fusion.shape}")

print("\n--- 8. FINAL EVALUATION (18 -> 17) ---")
metrics = evaluation.evaluate_model(
    rf_model, 
    X_test_fusion, 
    y_test, 
    save_dir=exp_paths['figures'],
    dataset_name="Cross-Dataset (Train 18 - Test 17)"
)

print(f"\nAccuracy: {metrics['accuracy']:.4f}")
print(f"MCC:      {metrics['mcc']:.4f}") # Expect significant drop compared to within-dataset [cite: 663]

📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_154439
📂 Results will be saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_154439

--- 1. LOADING TRAINING SET (CSE-CIC-IDS2018) ---
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)
Training Data Shape: (1000000, 65)

--- 2. SCALING & FEATURE SELECTION ---
✅ Scaler fitted on 2018 and saved.

--- 3. TRAINING AUTOENCODER ---
[Autoencoder] Training on cuda...
  Epoch 5/50 - Loss: 0.491886
  Epoch 10/50 - Loss: 0.341696
  Epoch 15/50 - Loss: 0.296804
  Epoch 20/50 - Loss: 0.260369
  Epoch 25/50 - Loss: 0.239797
  Epoch 30/50 - Loss: 0.216997
  Epoch 35/50 - Loss: 0.200022
  Epoch 40/50 - Loss: 0.182248
  Epoch 45/50 - Loss: 0.169419
  Epoch 50/50 - Loss: 0.156728
✅ AE Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_154439\models\ae_model_2018.pth
Extracting latent space features...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100building tree 10 of 100

building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   14.0s


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   46.5s finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_154439\models\rf_cross_18_to_17.joblib
✅ Hybrid Model trained on 2018.

--- 6. LOADING TEST SET (CIC-IDS2017) ---
🔄 Loading dataset year 2017 (Binary=True)...
✅ Loaded 2017. Shape: (2830743, 65)
Test Set Shape: (2830743, 65)

--- 7. PROCESSING TEST DATA ---
Test Fusion Shape: (2830743, 25)

--- 8. FINAL EVALUATION (18 -> 17) ---

📊 Evaluating on Cross-Dataset (Train 18 - Test 17)...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    2.4s finished


   ✅ Accuracy: 0.8378
   ⭐ MCC:      0.3765
   📝 Report saved to: report_Cross-Dataset_(Train_18_-_Test_17).txt
   🖼️ Confusion Matrix saved to: cm_Cross-Dataset_(Train_18_-_Test_17).png

Accuracy: 0.8378
MCC:      0.3765
